# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 28 2022 1:10PM,240645,12,HH-35533,Hush Hush,Released
1,Jun 28 2022 1:10PM,240645,12,HH-35534,Hush Hush,Released
2,Jun 28 2022 1:10PM,240645,12,HH-35535,Hush Hush,Released
3,Jun 28 2022 1:10PM,240644,19,ACG-2102490464,ACG North America LLC,Released
4,Jun 28 2022 12:53PM,240643,20,8173919-FLS,"ACI Healthcare USA, Inc.",Released
5,Jun 28 2022 12:53PM,240643,20,8187217,"ACI Healthcare USA, Inc.",Released
6,Jun 28 2022 12:53PM,240643,20,8199647,"ACI Healthcare USA, Inc.",Released
7,Jun 28 2022 12:53PM,240643,20,8201050,"ACI Healthcare USA, Inc.",Released
8,Jun 28 2022 12:53PM,240643,20,8201051,"ACI Healthcare USA, Inc.",Released
9,Jun 28 2022 12:53PM,240643,20,8201052,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,240641,Released,1
48,240642,Released,8
49,240643,Released,17
50,240644,Released,1
51,240645,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240641,NaN,1.0
240642,NaN,8.0
240643,NaN,17.0
240644,NaN,1.0
240645,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240550,0.0,1.0
240574,0.0,1.0
240577,0.0,1.0
240578,2.0,0.0
240579,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240550,0,1
240574,0,1
240577,0,1
240578,2,0
240579,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240550,0,1
1,240574,0,1
2,240577,0,1
3,240578,2,0
4,240579,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240550,,1
1,240574,,1
2,240577,,1
3,240578,2,
4,240579,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 28 2022 1:10PM,240645,12,Hush Hush
3,Jun 28 2022 1:10PM,240644,19,ACG North America LLC
4,Jun 28 2022 12:53PM,240643,20,"ACI Healthcare USA, Inc."
21,Jun 28 2022 12:52PM,240642,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
29,Jun 28 2022 12:51PM,240641,10,"Senseonics, Incorporated"
30,Jun 28 2022 12:43PM,240639,10,"Methapharm, Inc."
36,Jun 28 2022 12:43PM,240640,10,"Methapharm, Inc."
42,Jun 28 2022 12:37PM,240638,22,"NBTY Global, Inc."
43,Jun 28 2022 12:33PM,240637,10,MedStone Pharma LLC
48,Jun 28 2022 12:30PM,240636,10,Beach Patient Assistance


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 28 2022 1:10PM,240645,12,Hush Hush,,3
1,Jun 28 2022 1:10PM,240644,19,ACG North America LLC,,1
2,Jun 28 2022 12:53PM,240643,20,"ACI Healthcare USA, Inc.",,17
3,Jun 28 2022 12:52PM,240642,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,8
4,Jun 28 2022 12:51PM,240641,10,"Senseonics, Incorporated",,1
5,Jun 28 2022 12:43PM,240639,10,"Methapharm, Inc.",,6
6,Jun 28 2022 12:43PM,240640,10,"Methapharm, Inc.",,6
7,Jun 28 2022 12:37PM,240638,22,"NBTY Global, Inc.",,1
8,Jun 28 2022 12:33PM,240637,10,MedStone Pharma LLC,,5
9,Jun 28 2022 12:30PM,240636,10,Beach Patient Assistance,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 28 2022 1:10PM,240645,12,Hush Hush,3,
1,Jun 28 2022 1:10PM,240644,19,ACG North America LLC,1,
2,Jun 28 2022 12:53PM,240643,20,"ACI Healthcare USA, Inc.",17,
3,Jun 28 2022 12:52PM,240642,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8,
4,Jun 28 2022 12:51PM,240641,10,"Senseonics, Incorporated",1,
5,Jun 28 2022 12:43PM,240639,10,"Methapharm, Inc.",6,
6,Jun 28 2022 12:43PM,240640,10,"Methapharm, Inc.",6,
7,Jun 28 2022 12:37PM,240638,22,"NBTY Global, Inc.",1,
8,Jun 28 2022 12:33PM,240637,10,MedStone Pharma LLC,5,
9,Jun 28 2022 12:30PM,240636,10,Beach Patient Assistance,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 28 2022 1:10PM,240645,12,Hush Hush,3,
1,Jun 28 2022 1:10PM,240644,19,ACG North America LLC,1,
2,Jun 28 2022 12:53PM,240643,20,"ACI Healthcare USA, Inc.",17,
3,Jun 28 2022 12:52PM,240642,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8,
4,Jun 28 2022 12:51PM,240641,10,"Senseonics, Incorporated",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 28 2022 1:10PM,240645,12,Hush Hush,3.0,NaN
1,Jun 28 2022 1:10PM,240644,19,ACG North America LLC,1.0,NaN
2,Jun 28 2022 12:53PM,240643,20,"ACI Healthcare USA, Inc.",17.0,NaN
3,Jun 28 2022 12:52PM,240642,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8.0,NaN
4,Jun 28 2022 12:51PM,240641,10,"Senseonics, Incorporated",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 28 2022 1:10PM,240645,12,Hush Hush,3.0,0.0
1,Jun 28 2022 1:10PM,240644,19,ACG North America LLC,1.0,0.0
2,Jun 28 2022 12:53PM,240643,20,"ACI Healthcare USA, Inc.",17.0,0.0
3,Jun 28 2022 12:52PM,240642,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8.0,0.0
4,Jun 28 2022 12:51PM,240641,10,"Senseonics, Incorporated",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2887514,89.0,17.0
12,240645,3.0,0.0
15,1684300,52.0,52.0
16,240594,0.0,7.0
17,240550,1.0,0.0
18,240614,1.0,0.0
19,1684268,31.0,9.0
20,1443745,66.0,12.0
21,1203047,5.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2887514,89.0,17.0
1,12,240645,3.0,0.0
2,15,1684300,52.0,52.0
3,16,240594,0.0,7.0
4,17,240550,1.0,0.0
5,18,240614,1.0,0.0
6,19,1684268,31.0,9.0
7,20,1443745,66.0,12.0
8,21,1203047,5.0,0.0
9,22,962516,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,89.0,17.0
1,12,3.0,0.0
2,15,52.0,52.0
3,16,0.0,7.0
4,17,1.0,0.0
5,18,1.0,0.0
6,19,31.0,9.0
7,20,66.0,12.0
8,21,5.0,0.0
9,22,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,89.0
1,12,Released,3.0
2,15,Released,52.0
3,16,Released,0.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,
Executing,17.0,0.0,52.0,7.0,0.0,0.0,9.0,12.0,0.0,0.0
Released,89.0,3.0,52.0,0.0,1.0,1.0,31.0,66.0,5.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21,22
0,Executing,17.0,0.0,52.0,7.0,0.0,0.0,9.0,12.0,0.0,0.0
1,Released,89.0,3.0,52.0,0.0,1.0,1.0,31.0,66.0,5.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21,22
0,Executing,17.0,0.0,52.0,7.0,0.0,0.0,9.0,12.0,0.0,0.0
1,Released,89.0,3.0,52.0,0.0,1.0,1.0,31.0,66.0,5.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()